In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
'''
under path:

benchmarkdataset_train.fasta
benchmarkdataset_test.fasta

NT15dataset_train.fasta 
NT15dataset_test.fasta
'''

path = '/home/yochen/bioinformatics/AntiAngiogenic/'

In [ ]:
def get_data(filename):
    with open(path + filename) as f:
        lines = f.readlines()

    for i in range(len(lines)):
        lines[i] = lines[i].strip()
    
    return lines

In [ ]:
get_data('benchmarkdataset_train.fasta')

In [ ]:
def clean_raw_data(raw_data):
    clean = []
    
    for i in range(0, len(raw_data), 2):
        if raw_data[i][1] == 'n': clean.append([raw_data[i + 1], 0])
        else: clean.append([raw_data[i + 1], 1])
        
    return clean

In [ ]:
clean_raw_data(get_data('benchmarkdataset_train.fasta'))

In [ ]:
def get_aac(seq, aa):
    aac = 0
    for amino_acid in seq:
        if amino_acid == aa: aac += 1
    aac /= len(seq)
    return aac

In [ ]:
get_aac('ARNNR', 'A')

In [ ]:
def get_dpc(seq, dp):
    dpc = 0
    for i in range(0, len(seq) - 1):
         if seq[i:i + 2] == dp: dpc += 1
    dpc /= (len(seq) - 1)
    return dpc

In [ ]:
get_dpc('ARNNR', 'AR')

In [ ]:
def get_tc(seq, tp):
    tc = 0
    for i in range(0, len(seq) - 2):
         if seq[i:i + 3] == tp: tc += 1
    tc /= (len(seq) - 2)
    return tc

In [ ]:
get_tc('ARNNR', 'ARN')

In [ ]:
df = pd.read_csv('aaindex1.csv')
df = df.drop('Description', axis='columns')
df = df.fillna(0)
df

In [ ]:
aa_props = df.to_numpy()

mean = np.mean(aa_props, axis=1).reshape((566, 1))
std = np.std(aa_props, axis=1).reshape((566, 1))

aa_props -= mean
aa_props /= std

aa_props[0]

In [ ]:
aa_list = list(df.columns)
aa_to_index = {}
for i, aa in enumerate(aa_list): aa_to_index[aa] = i

print(aa_list)    
print(aa_to_index)

In [ ]:
def correlation(ri, rj):
    ans = 0
    i = aa_to_index[ri]
    j = aa_to_index[rj]
    for prop in aa_props: ans += (prop[i] - prop[j]) ** 2
    ans /= len(aa_props)
    return ans

In [ ]:
def correlation_angle(ri, rj):
    i = aa_to_index[ri]
    j = aa_to_index[rj]
    
    vi = aa_props[:, i]
    vj = aa_props[:, j]
    
    cos = np.dot(vi, vj) / (np.linalg.norm(vi) * np.linalg.norm(vj))
    angle = np.arccos(np.clip(cos, -1, 1))
    
    return angle

In [ ]:
def get_pseaac(seq, lam=10, w=0.05):
    pseaac = []
    n = len(seq)
    
    for aa in aa_list: pseaac.append(get_aac(seq, aa))
        
    for i in range(1, lam + 1):
        theta_i = 0
        for j in range(i, n): theta_i += correlation(seq[j - i], seq[j])
        theta_i /= (n - i)
        pseaac.append(w * theta_i)
        
    pseaac = np.array(pseaac)
    pseaac /= (1 + w * np.sum(pseaac[20:]))
    
    return pseaac

In [ ]:
get_pseaac('IVDDWIYMIEEICKI')

In [ ]:
def get_pseaac_angle(seq, lam=10, w=0.05):
    pseaac = []
    n = len(seq)
    
    for aa in aa_list: pseaac.append(get_aac(seq, aa))
        
    for i in range(1, lam + 1):
        theta_i = 0
        for j in range(i, n): theta_i += correlation_angle(seq[j - i], seq[j]) # use a differnt correlation function
        theta_i /= (n - i)
        pseaac.append(w * theta_i)
        
    pseaac = np.array(pseaac)
    pseaac /= (1 + w * np.sum(pseaac[20:]))
    
    return pseaac

In [ ]:
get_pseaac_angle('IVDDWIYMIEEICKI')

In [ ]:
def seq2vec(seq):
    vector = []
    
    for aa in aa_list: vector.append(get_aac(seq, aa))
        
    for aa1 in aa_list:
        for aa2 in aa_list:
            vector.append(get_dpc(seq, aa1 + aa2))
            
    for aa1 in aa_list:
        for aa2 in aa_list:
            for aa3 in aa_list:
                vector.append(get_dpc(seq, aa1 + aa2 + aa3))
    
    vector += list(get_pseaac(seq))
    vector += list(get_pseaac_angle(seq))
    
    vector = np.array(vector)
            
    return vector

In [ ]:
print(len(seq2vec('KRFKQDGGWSHWSPWSSCSVTCGDGVITRIRLCNSPSPQMNGKPCEGEARETKACKKDACPI')))
print(seq2vec('KRFKQDGGWSHWSPWSSCSVTCGDGVITRIRLCNSPSPQMNGKPCEGEARETKACKKDACPI'))

In [ ]:
def generate_xy(data):
    x = []
    y = []
    
    for pair in data:
        x.append(seq2vec(pair[0]))
        y.append(pair[1])
    
    x = np.array(x)
    y = np.array(y)
    
    return x, y

In [ ]:
x_train, y_train = generate_xy(clean_raw_data(get_data('benchmarkdataset_train.fasta')))

print(x_train)
print()
print(y_train)

In [ ]:
print(len(x_train))
print(len(y_train))

In [ ]:
np.save('benchmarkdataset_x_train.npy', x_train)
np.save('benchmarkdataset_y_train.npy', y_train)

In [ ]:
x_test, y_test = generate_xy(clean_raw_data(get_data('benchmarkdataset_test.fasta')))

print(x_test)
print()
print(y_test)

In [ ]:
print(len(x_test))
print(len(y_test))

In [ ]:
np.save('benchmarkdataset_x_test.npy', x_test)
np.save('benchmarkdataset_y_test.npy', y_test)

In [ ]:
x_train, y_train = generate_xy(clean_raw_data(get_data('NT15dataset_train.fasta')))

print(x_train)
print()
print(y_train)

In [ ]:
print(len(x_train))
print(len(y_train))

In [ ]:
np.save('NT15dataset_x_train.npy', x_train)
np.save('NT15dataset_y_train.npy', y_train)

In [ ]:
x_test, y_test = generate_xy(clean_raw_data(get_data('NT15dataset_test.fasta')))

print(x_test)
print()
print(y_test)

In [ ]:
print(len(x_test))
print(len(y_test))

In [ ]:
np.save('NT15dataset_x_test.npy', x_test)
np.save('NT15dataset_y_test.npy', y_test)